In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import pandas_datareader.data as web

from pathlib import Path
import numpy as np
import pandas as pd
import gc

import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

with pd.HDFStore(DATA_STORE) as store:
    data = store['data/top500_dataset']

In [4]:
selected_data = data[['open', 'high', 'low', 'close', 'volume']]

In [5]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1336327 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZION', Timestamp('2023-08-10 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1336327 non-null  float64
 1   high    1336327 non-null  float64
 2   low     1336327 non-null  float64
 3   close   1336327 non-null  float64
 4   volume  1336327 non-null  float64
dtypes: float64(5)
memory usage: 56.5+ MB


In [6]:
# import numpy as np
# import matplotlib.pyplot as plt
# from tsfracdiff import FractionalDifferentiator

# def process_tickers(df):
#     all_orders = []

#     for ticker, group in df.groupby(level=0):
#         group = group.reset_index(drop=True)  # Reset the multi-index for the current group

#         # Focus on the 'close' prices
#         X_ticker = group[['close']]
#         # print(X_ticker)
#         # break

#         # Calculate percentage change (returns)
#         X_pct_change = X_ticker.pct_change().dropna()

#         # 1. Fractional Differentiation for the specific ticker
#         fracDiff = FractionalDifferentiator()
#         X_frac = fracDiff.FitTransform(X_ticker)

#         # Print estimated orders for the specific ticker
#         orders = fracDiff.orders
#         print(f"For ticker {ticker}, estimated orders range from {min(orders)} to {max(orders)}.")
#         all_orders.extend(orders)

#         # 2. First Difference
#         X_diff = X_ticker.diff().dropna()

#         # 3. Plotting
#         fig, axes = plt.subplots(3, 1, figsize=(15, 12))
        
#         X_diff.plot(title=f'{ticker} - Differenced Prices', ax=axes[0], color='blue')
#         axes[0].set_ylabel('Differenced Close')
        
#         X_ticker.plot(ax=axes[1], label='Original Close', color='blue')
#         X_frac.plot(title=f'{ticker} - Fractionally Differentiated Prices vs Original Close', ax=axes[1], linestyle='--', color='orange')
#         axes[1].set_ylabel('Close Prices')
#         axes[1].legend()
        
#         X_pct_change.plot(title=f'{ticker} - Percentage Change (Returns)', ax=axes[2], color='green')
#         axes[2].set_ylabel('Percentage Change')

#         plt.tight_layout()
#         plt.show()

#         # 4. Plotting Correlations
#         fig, ax = plt.subplots(figsize=(12, 6))
#         correlations = {
#             'Differenced Close': X_diff.corrwith(X_ticker)['close'],
#             'Fractional Difference': X_frac.corrwith(X_ticker)['close'],
#             'Percentage Change': X_pct_change.corrwith(X_ticker).iloc[0]
#         }
        
#         ax.bar(correlations.keys(), correlations.values(), alpha=0.7)
#         ax.set_title(f'{ticker} - Correlation of Transformed Features with Original')
#         ax.set_ylabel('Correlation Coefficient')
#         plt.tight_layout()
#         plt.show()
#         # break

#     # Analyzing orders after processing all tickers
#     print(f"Across all tickers, estimated orders range from {min(all_orders)} to {max(all_orders)}.")

# # Assuming df is your DataFrame
# process_tickers(selected_data)

In [7]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from tsfracdiff import FractionalDifferentiator

# def process_tickers(df):
#     processed_dfs = []  # Container to store modified dataframes
#     i = 0
#     for ticker, group in df.groupby(level=0):
#         group = group.copy().reset_index(drop=True)  # Reset the multi-index for the current group

#         # Calculate X_frac (I'm assuming you have a method to do this)
#         X_frac = FractionalDifferentiator().FitTransform(group[['close']])

#         # The number of NaN values needed to align X_frac with close
#         nans_needed = len(group) - len(X_frac)

#         # Create a NaN series of required length
#         nan_series = pd.Series([np.nan] * nans_needed)

#         # Append the NaN series to the start of X_frac
#         X_frac_aligned = nan_series.append(X_frac['close']).reset_index(drop=True)

#         # Add the aligned X_frac to the group
#         group['X_frac'] = X_frac_aligned

#         processed_dfs.append(group)
#         if (i > 5):
#             break
#         i += 1

#     # Concatenate all processed dataframes
#     result_df = pd.concat(processed_dfs)

#     return result_df

# updated = process_tickers(selected_data)

In [8]:
import numpy as np
import pandas as pd
from tsfracdiff import FractionalDifferentiator

def process_tickers(df, num_tickers=None):
    processed_dfs = []

    # If num_tickers is not provided, process all tickers
    if num_tickers is None:
        ticker_groups = list(df.groupby(level=0))
    else:
        ticker_groups = list(df.groupby(level=0))[:num_tickers]

    for ticker, group in ticker_groups:
        group_copy = group.copy()

        # Calculate X_frac 
        frac_diff = FractionalDifferentiator()
        X_frac = frac_diff.FitTransform(group_copy[['close']])
    
        # The number of NaN values needed to align X_frac with close
        nans_needed = len(group_copy) - len(X_frac)

        # Create a NaN series of required length
        nan_series = pd.Series([np.nan] * nans_needed)

        # Append the NaN series to the start of X_frac
        X_frac_aligned = nan_series.append(X_frac['close']).reset_index(drop=True)

        # Add the aligned X_frac to the group
        column_name = f"ret_frac_order"
        group_copy[column_name] = X_frac_aligned.values

        processed_dfs.append(group_copy)

    # Concatenate all processed dataframes
    result_df = pd.concat(processed_dfs)

    return result_df

# Process all tickers
updated = process_tickers(data)

In [9]:
from utils import optimize_dataframe
op = optimize_dataframe(updated)

Data memory before optimization: 209.13 MB
Data memory after optimization: 117.37 MB
Reduced by: 43.88%


In [10]:
op.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1336327 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-10 00:00:00'))
Data columns (total 20 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   open            1336327 non-null  float32
 1   high            1336327 non-null  float32
 2   low             1336327 non-null  float32
 3   close           1336327 non-null  float32
 4   volume          1336327 non-null  float64
 5   market cap      1336327 non-null  float64
 6   sector          1336327 non-null  float32
 7   ret_01d         1336327 non-null  float32
 8   ret_02d         1336327 non-null  float32
 9   ret_03d         1336327 non-null  float32
 10  ret_04d         1336327 non-null  float32
 11  ret_05d         1336327 non-null  float32
 12  ret_10d         1336327 non-null  float32
 13  ret_21d         1336327 non-null  float32
 14  ret_42d         1336327 non-null  float32
 15  ret_63d   

In [11]:
# # ### Parallel code
# import numpy as np
# import pandas as pd
# from tsfracdiff import FractionalDifferentiator
# from concurrent.futures import ProcessPoolExecutor

# def process_one_ticker(data_group):
#     ticker, group = data_group
#     group_copy = group.copy()

#     # Calculate X_frac 
#     frac_diff = FractionalDifferentiator()
#     X_frac = frac_diff.FitTransform(group_copy[['close']])
    
#     # The number of NaN values needed to align X_frac with close
#     nans_needed = len(group_copy) - len(X_frac)

#     # Create a NaN series of required length
#     nan_series = pd.Series([np.nan] * nans_needed)

#     # Append the NaN series to the start of X_frac
#     X_frac_aligned = nan_series.append(X_frac['close']).reset_index(drop=True)

#     # Add the aligned X_frac to the group
#     column_name = f"ret_frac_order"
#     group_copy[column_name] = X_frac_aligned.values

#     return group_copy

# def process_tickers_parallel(df, num_tickers=None, max_workers=20):
#     # If num_tickers is not provided, process all tickers
#     if num_tickers is None:
#         ticker_groups = list(df.groupby(level=0))
#     else:
#         ticker_groups = list(df.groupby(level=0))[:num_tickers]

#     with ProcessPoolExecutor(max_workers=max_workers) as executor:
#         processed_dfs = list(executor.map(process_one_ticker, ticker_groups))

#     # Concatenate all processed dataframes
#     result_df = pd.concat(processed_dfs)

#     return result_df

# # Usage:
# updated = process_tickers_parallel(selected_data)

In [12]:
updated

open        high         low       close  \
ticker date                                                         
AA     2013-01-03   18.904100   19.306499   18.883200   19.137600   
       2013-01-04   19.222000   19.579800   19.200300   19.537100   
       2013-01-07   19.537100   19.600599   19.200300   19.200300   
       2013-01-08   19.327400   19.389099   19.158400   19.200300   
       2013-01-09   19.685101   19.685101   19.115700   19.158400   
...                       ...         ...         ...         ...   
ZTS    2023-08-04  180.139999  182.460007  178.339996  178.630005   
       2023-08-07  180.190002  181.880005  180.029999  180.690002   
       2023-08-08  182.149994  190.539993  177.250000  189.300003   
       2023-08-09  189.940002  190.949997  186.964996  189.350006   
       2023-08-10  190.000000  191.824997  187.729996  189.100006   

                         volume    market cap  sector   ret_01d   ret_02d  \
ticker date                                                                 
AA     2013-01-03  1.057118e+07  5.703249e+09     4.0  0.020875  0.003276   
       2013-01-04  8.690739e+06  5.703249e+09     4.0  0.020875  0.003276   
       2013-01-07  1.031992e+07  5.703249e+09     4.0 -0.017239  0.003276   
       2013-01-08  1.609879e+07  5.703249e+09     4.0  0.000000 -0.017239   
       2013-01-09  1.979932e+07  5.703249e+09     4.0 -0.002182 -0.002182   
...                         ...           ...     ...       ...       ...   
ZTS    2023-08-04  1.827463e+06  8.818486e+10     2.0 -0.012439 -0.018516   
       2023-08-07  1.941699e+06  8.818486e+10     2.0  0.011532 -0.001050   
       2023-08-08  4.117986e+06  8.818486e+10     2.0  0.047651  0.059732   
       2023-08-09  2.270030e+06  8.818486e+10     2.0  0.000264  0.047927   
       2023-08-10  2.348639e+06  8.818486e+10     2.0 -0.001320 -0.001057   

                    ret_03d   ret_04d   ret_05d   ret_10d   ret_21d   ret_42d  \
ticker date                                                                     
AA     2013-01-03  0.003276  0.001087 -0.011062 -0.014385 -0.023211 -0.051981   
       2013-01-04  0.003276  0.001087 -0.011062 -0.014385 -0.023211 -0.051981   
       2013-01-07  0.003276  0.001087 -0.011062 -0.014385 -0.023211 -0.051981   
       2013-01-08  0.003276  0.001087 -0.011062 -0.014385 -0.023211 -0.051981   
       2013-01-09 -0.019384  0.001087 -0.011062 -0.014385 -0.023211 -0.051981   
...                     ...       ...       ...       ...       ...       ...   
ZTS    2023-08-04 -0.037087 -0.050295 -0.059347 -0.026593  0.060307  0.043887   
       2023-08-07 -0.007198 -0.025982 -0.039343 -0.026350  0.081070  0.062008   
       2023-08-08  0.046550  0.040110  0.020430 -0.004470  0.112090  0.157232   
       2023-08-09  0.060012  0.046827  0.040385 -0.013802  0.112907  0.152465   
       2023-08-10  0.046544  0.058613  0.045444  0.002173  0.100314  0.159482   

                    ret_63d  ret_126d  ret_252d  ret_fwd_01d  ret_frac_order  
ticker date                                                                   
AA     2013-01-03 -0.088433 -0.133089  0.181219     0.020875        8.914233  
       2013-01-04 -0.088433 -0.133089  0.181219    -0.017239        8.914233  
       2013-01-07 -0.088433 -0.133089  0.181219     0.000000        8.914233  
       2013-01-08 -0.088433 -0.133089  0.181219    -0.002182        8.914233  
       2013-01-09 -0.088433 -0.133089  0.181219    -0.012136        8.914233  
...                     ...       ...       ...          ...             ...  
ZTS    2023-08-04  0.000897  0.044620 -0.006386     0.011532       13.458624  
       2023-08-07 -0.029748  0.076818  0.038806     0.047651       16.669207  
       2023-08-08  0.023963  0.147760  0.086433     0.000264       24.922548  
       2023-08-09  0.027011  0.143625  0.090468    -0.001320       21.726665  
       2023-08-10  0.021389  0.166780  0.117672     0.009149       20.455498  

[1336327 rows x 20 columns]